# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [141]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [142]:
from sklearn.model_selection import train_test_split

X = adult_dt.drop(columns = 'income')
Y = adult_dt['income']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 42)

## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

- The random state is equivalent to setting a random seed. In computers, "random" numbers are generated through calculations, so they’re technically pseudo-random rather than truly random. Setting a seed provides an initial value for these calculations, ensuring that the same sequence of numbers is generated each time, given the same seed value.

- Setting a random seed is useful in machine learning because it enables reproducibility. As we often run programs multiple times during development and testing, using a fixed random seed ensures that we get repeatable results across each run.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [143]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler

numerical_columns = X.select_dtypes(include=['number']).columns.to_list()

categorical_columns = X.select_dtypes(exclude=['number']).columns.to_list()

pipeline_numeric = Pipeline([
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('standardizer', RobustScaler())
])

pipeline_categorical = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='infrequent_if_exist', drop='first'))
])

column_transformer = ColumnTransformer([
    ('num_transforms', pipeline_numeric, numerical_columns),
    ('cat_transforms', pipeline_categorical, categorical_columns)
])

column_transformer

ColumnTransformer(transformers=[('num_transforms',
                                 Pipeline(steps=[('imputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('standardizer',
                                                  RobustScaler())]),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat_transforms',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='infrequent_if_exist'))]),
                                 ['workclass', 'education', 'marital-status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native-country'])])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [144]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('preprocessing', column_transformer),
    ('classifier', RandomForestClassifier())
])

pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('standardizer',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='infrequent_if_exist'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier', RandomForestClassifier())])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [performance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [145]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

performance_metrics = ['neg_log_loss', 'roc_auc', 'accuracy', 'balanced_accuracy']

pipeline_cv = cross_validate(pipeline, X_train, Y_train, cv=5, scoring=performance_metrics, return_train_score=True)

pipeline_cv

c:\Users\nalca\.conda\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\nalca\.conda\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


{'fit_time': array([7.47834396, 7.35035038, 7.21486092, 7.23645806, 7.38079166]),
 'score_time': array([0.16914392, 0.15337324, 0.14522624, 0.14638281, 0.15033937]),
 'test_neg_log_loss': array([-0.34902841, -0.35165806, -0.36353288, -0.37729402, -0.40902131]),
 'train_neg_log_loss': array([-0.08127099, -0.08161921, -0.08143828, -0.08201177, -0.08139319]),
 'test_roc_auc': array([0.90465952, 0.90364709, 0.90458045, 0.90671251, 0.90094586]),
 'train_roc_auc': array([1., 1., 1., 1., 1.]),
 'test_accuracy': array([0.84887037, 0.84777363, 0.85607723, 0.85936814, 0.85870996]),
 'train_accuracy': array([0.99994515, 1.        , 0.99989031, 1.        , 0.99994516]),
 'test_balanced_accuracy': array([0.77117495, 0.76860727, 0.78008919, 0.78164584, 0.77936531]),
 'train_balanced_accuracy': array([0.9999638 , 1.        , 0.99977386, 1.        , 0.99988693])}

Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [146]:
cv_results = pd.DataFrame(pipeline_cv).sort_values(by=['test_neg_log_loss'])

cv_results

,fit_time,score_time,test_neg_log_loss,train_neg_log_loss,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_balanced_accuracy,train_balanced_accuracy
4,7.380792,0.150339,-0.409021,-0.081393,0.900946,1.0,0.858710,0.999945,0.779365,0.999887
3,7.236458,0.146383,-0.377294,-0.082012,0.906713,1.0,0.859368,1.000000,0.781646,1.000000
2,7.214861,0.145226,-0.363533,-0.081438,0.904580,1.0,0.856077,0.999890,0.780089,0.999774
1,7.350350,0.153373,-0.351658,-0.081619,0.903647,1.0,0.847774,1.000000,0.768607,1.000000
0,7.478344,0.169144,-0.349028,-0.081271,0.904660,1.0,0.848870,0.999945,0.771175,0.999964


Calculate the mean of each metric. 

In [147]:
cv_results.mean()

fit_time                   7.332161
score_time                 0.152893
test_neg_log_loss         -0.370107
train_neg_log_loss        -0.081547
test_roc_auc               0.904109
train_roc_auc              1.000000
test_accuracy              0.854160
train_accuracy             0.999956
test_balanced_accuracy     0.776177
train_balanced_accuracy    0.999925
dtype: float64

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [150]:
pipeline.fit(X_train, Y_train)

Y_pred_test = pipeline.predict(X_test)
Y_proba_test = pipeline.predict_proba(X_test)

test_res = {
    'neg_loss_test': -log_loss(Y_test, Y_proba_test),
    'roc_auc_test': roc_auc_score(Y_test, Y_proba_test[:, -1]),
    'accuracy_score_test': accuracy_score(Y_test, Y_pred_test),
    'balanced_accuracy_test': balanced_accuracy_score(Y_test, Y_pred_test)
}

display(test_res)

{'neg_loss_test': -0.3819539497824786,
 'roc_auc_test': 0.901285964128186,
 'accuracy_score_test': 0.8534138601699253,
 'balanced_accuracy_test': 0.7725329215280157}

# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

- I believe it would depend on the results we want from our ML system, in this it seems like we are trying to make a binary classifier, and we only want to know if their incomes pass a certain threshold and nothing more, instead of trying to predict the values using regression models. Preprocessing simplifies our data and would most likely help as well with the performance of the model.

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.